In [42]:
from sqlalchemy import create_engine, Table, MetaData, select, func, case
import pandas as pd
import json

with open('parameters.json', 'r') as file:
    data = json.load(file)
    
# Define the database connection string (update with your PostgreSQL details)
DATABASE_URI = data["DATABASE_URI"]

# Create an engine and metadata object
engine = create_engine(DATABASE_URI)

# Create an engine and metadata object
metadata = MetaData()

# Reflect tables
t_financials = Table('t_financials', metadata, autoload_with=engine)
t_doc_list = Table('t_doc_list', metadata, autoload_with=engine)
t_edinet_code_mappings = Table('t_edinet_code_mappings', metadata, autoload_with=engine)
t_short_list = Table('t_short_list', metadata, autoload_with=engine)

# Define the CTE
v_summary_financials = (
    select(
        t_financials.c.docID,
        t_edinet_code_mappings.c['証券コード'].label('securities_code'),
        func.sum(case((t_financials.c.itemName.like('流動資産%'), t_financials.c.amount), else_=0)).label('liquid_assets'),
        func.sum(case((t_financials.c.itemName == '投資有価証券', t_financials.c.amount), else_=0)).label('investment_securities'),
        func.sum(case((t_financials.c.itemName == '負債', t_financials.c.amount), else_=0)).label('liabilities')
    )
    .select_from(
        t_financials.join(t_doc_list, t_financials.c.docID == t_doc_list.c.docID)
                    .join(t_edinet_code_mappings, t_doc_list.c.edinetCode == t_edinet_code_mappings.c['ＥＤＩＮＥＴコード'])
    )
    .where(t_financials.c.categoryID == 'CurrentYearInstant')
    .group_by(t_financials.c.docID, t_edinet_code_mappings.c['証券コード'])
    .cte('v_summary_financials')
)

# Now define the main query
main_query = (
    select(
        t_short_list,
        v_summary_financials.c.liquid_assets,
        v_summary_financials.c.investment_securities,
        v_summary_financials.c.liabilities
    )
    .select_from(
        t_short_list.outerjoin(v_summary_financials, t_short_list.c.Code_x == v_summary_financials.c.securities_code)
    )
)

# Execute the query and fetch the results
with engine.connect() as connection:
    df = pd.read_sql(main_query, connection)

query_edinet_mappings = "select * from t_edinet_code_mappings"

In [45]:
query_edinet_mappings = "select * from t_edinet_code_mappings"
with engine.connect() as connection:
    df_edinet = pd.read_sql(query_edinet_mappings, connection)

In [46]:
df["net_cash"] = df["liquid_assets"] - df["liabilities"] + (df["investment_securities"] * .7)

In [47]:
df["net_cash_ratio"] = df["net_cash"]/df["marketCap"]

In [48]:
#df.head()

columns_string = "Code_x, CompanyName, CompanyNameEnglish, Sector17CodeName, marketCap, previousClose, trailingPE, net_cash_ratio, net_cash"

# Convert string to list
columns_list = columns_string.split(", ")

df2 = df[columns_list]

In [56]:
df2['Code_x'] = df2['Code_x'].astype(str)
df_edinet_renamed = df_edinet.rename(columns={"証券コード": "Code_x"})
df_edinet_renamed['Code_x'] = df_edinet_renamed['Code_x'].astype(str)

result = df2.join(df_edinet_renamed.set_index('Code_x'), on='Code_x')

/var/folders/4d/73_4mvkd4nsc69k9dpv182fr0000gn/T/ipykernel_59180/447275019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Code_x'] = df2['Code_x'].astype(str)


In [61]:
edinet_code_list = result[result["net_cash"].isna()]["ＥＤＩＮＥＴコード"].values

In [63]:
edinet_code_list

array(['E03432', 'E03746', 'E02762', 'E03093', 'E30124', 'E37141',
       'E27202', 'E04064', 'E35544', 'E27530', 'E38443'], dtype=object)

In [49]:
df2[df2["net_cash"].isna()].join()

,Code_x,CompanyName,CompanyNameEnglish,Sector17CodeName,marketCap,previousClose,trailingPE,net_cash_ratio,net_cash
558,33170,フライングガーデン,"FLYING GARDEN CO.,LTD.",小売,3757467904,2601,9.478673,NaN,NaN
559,87720,アサックス,"ASAX CO.,LTD.",金融（除く銀行）,26314848256,799,8.000000,NaN,NaN
560,99410,太洋物産,"TAIYO BUSSAN KAISHA,LTD.",商社・卸売,1944869632,1004,9.174646,NaN,NaN
561,81660,タカキュー,"Taka-Q Co.,Ltd.",小売,3118732800,132,2.815043,NaN,NaN
562,32940,イーグランド,"e'grand Co.,Ltd",不動産,9530742784,1570,7.639464,NaN,NaN
563,29930,長栄,Choei Inc.,不動産,8952217600,2040,7.209745,NaN,NaN
564,31810,買取王国,"KAITORI OKOKU CO.,LTD.",小売,3087454976,852,8.183306,NaN,NaN
565,32360,プロパスト,"PROPERST CO.,LTD.",不動産,6417041408,188,3.526120,NaN,NaN
566,44940,バリオセキュア,Vario Secure Inc.,情報通信・サービスその他,3054526208,680,8.787209,NaN,NaN
567,62480,横田製作所,"Yokota Manufacturing Co., Ltd.",機械,2554527744,1385,9.434092,NaN,NaN


In [24]:
df2[df2["net_cash_ratio"] >= 1].to_csv("short_list.csv", index=False)